In [1]:
import torch
from torch import nn
from src.utils.preprocess_utils import notes_to_midi, midi_to_note_indices, one_hot_encode
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [2]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, input_size]
        out, _ = self.lstm(x)  # out: [batch_size, sequence_length, hidden_size]
        out = self.fc(out)
        return out

In [3]:
# Load model
model_name = "2025-01-03_19-34-53_h128_l2_sl150"
model = torch.load(f'weights/model_1/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_19756\371218633.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_1/{model_name}')


LSTMModel(
  (lstm): LSTM(85, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [4]:
# Load input
input = midi_to_note_indices('data/input.mid', 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [5]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_sequence_one_hot = one_hot_encode(start_sequence, num_notes=num_notes)
    start_tensor = torch.tensor(start_sequence_one_hot, dtype=torch.float32).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(one_hot_encode(generated_sequence_tensor, num_notes=num_notes), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 51, 46, 35, 45, 45, 48, 45, 52, 38, 45, 52, 29, 47, 36, 38, 47, 40, 37, 40, 45, 43, 46, 43, 36, 50, 33, 38, 50, 33, 34, 31, 36, 43, 59, 33, 31, 36, 50, 38, 36, 36, 45, 47, 26, 40, 31, 36, 51, 30, 49, 39, 43, 43, 59, 45, 36, 36, 45, 50, 45, 58, 45, 36, 50, 43, 52, 29, 43, 46, 63, 38, 31, 41, 47, 42, 36, 45, 36, 45, 23, 50, 52, 50, 50, 47, 31, 38, 43, 48, 43, 50, 34, 38, 42, 39, 39, 40, 31, 43, 35, 31, 34, 36, 52, 38, 33, 52, 35, 38, 40, 38, 33, 51, 38, 34, 46, 52, 48, 42, 31, 34, 43, 29, 55, 31, 46, 52, 45, 40, 53, 33, 51, 45, 44, 50, 40, 48, 50, 49, 48, 45, 47, 28, 44, 57, 35, 40, 42, 49, 46, 53, 33, 30, 37, 47, 38, 36, 52, 43, 48, 50, 39, 50, 47, 50, 38, 25, 33, 38, 41, 31, 40, 36, 48, 38, 40, 45, 43, 45, 50, 45, 46, 50, 35, 48, 31, 48, 45, 51, 36, 54, 50, 47, 50, 26, 38, 45, 42, 54, 43]


In [6]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_1/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_1/2025-01-04_09-32-56_t0.3_w200_M_2025-01-03_19-34-53_h128_l2_sl150.mid
